In [36]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load original data set
data_path = '/Volumes/Transcend/metis_data/Turnstile_Usage_Data__2018.csv'
df_orig = pd.read_csv(data_path )
df_orig

In [ ]:
import re
# Truncating data set
mask = df_orig['Date'].map(lambda x: re.search('(05|06)/.+/2018', x) != None)
df = df_orig[mask]

# Adding new column 'Full Time' with data and time information
df['Full Time']=pd.to_datetime(df['Date'] + ' ' + df['Time'], format = '%m/%d/%Y %H:%M:%S')

# Adding new column for day of week
df['dayofweek']=df['Full Time'].dt.dayofweek


In [91]:
# Creates list of times we want to query.
may = []
june = []
for i in range(1,32):
    may.append('2018-05-' + '{:02d}'.format(i)+ ' 00:00:00')
for i in range(1,31):
    june.append('2018-06-' + '{:02d}'.format(i) + ' 00:00:00')
times_query = may + june

# Create a group dataframe for each individual turnstile
gdf = df.groupby(['SCP','C/A','Station'])

# Initialize empty dataframe to append entries per day 
df_day = pd.DataFrame()

# iterates over turnstiles
for name, group in gdf:
    
    # filter out only relevant columns
    df_dev = group[['SCP','C/A','Entries','Full Time','Station']]
    
    # Place nan's at time points where we want to interpolate
    for time in times_query:    
        if df_dev[df_dev['Full Time']==time].empty:
            df_newrow = pd.DataFrame.from_dict({'Full Time': pd.Timestamp(time), 
                                                'Entries': [np.nan],
                                                 'SCP':name[0],
                                                 'C/A':name[1],
                                                 'Station':name[2]})
            df_dev = pd.concat([df_dev,df_newrow])
    
    # Interpolate all numerical values ('Entries' is the only numerical column in this case)
    df_dev = df_dev.sort_values(by=['Full Time'])
    df_dev = df_dev.interpolate()
    
    # Filters on just the times we queried in times_query list
    mask = df_dev['Full Time'].map(lambda x: str(x) in times_query)
    df_dev = df_dev[mask]
    
    # Finds the time difference between days
    df_dev = df_dev.sort_values(by=['Full Time'])
    df_dev['Entries_diff']=df_dev['Entries'].diff()
    
    # Appends entries per day data for each turnstile.
    df_day = pd.concat([df_day,df_dev])
    
print(df_day)

/Users/cmiless/miniconda3/envs/home/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/cmiless/miniconda3/envs/home/lib/python3.6/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



     C/A    Entries  Entries_diff  Full Time       SCP     Station
0   A006        NaN           NaN 2018-05-01  00-00-00  5 AV/59 ST
0   A006  3328106.5           NaN 2018-05-02  00-00-00  5 AV/59 ST
0   A006  3329790.5        1684.0 2018-05-03  00-00-00  5 AV/59 ST
0   A006  3331426.5        1636.0 2018-05-04  00-00-00  5 AV/59 ST
0   A006  3333145.0        1718.5 2018-05-05  00-00-00  5 AV/59 ST
0   A006  3334095.0         950.0 2018-05-06  00-00-00  5 AV/59 ST
0   A006  3334799.0         704.0 2018-05-07  00-00-00  5 AV/59 ST
0   A006  3336401.0        1602.0 2018-05-08  00-00-00  5 AV/59 ST
0   A006  3338129.5        1728.5 2018-05-09  00-00-00  5 AV/59 ST
0   A006  3339850.5        1721.0 2018-05-10  00-00-00  5 AV/59 ST
0   A006  3341488.5        1638.0 2018-05-11  00-00-00  5 AV/59 ST
0   A006  3343195.0        1706.5 2018-05-12  00-00-00  5 AV/59 ST
0   A006  3344108.5         913.5 2018-05-13  00-00-00  5 AV/59 ST
0   A006  3344810.0         701.5 2018-05-14  00-00-00  5 AV/5

In [101]:
# Postprocessing
df_day_new = df_day.reset_index(drop=True)
df_day_new = df_day_new.dropna() # removes nan's due to interpolation at end cases
df_day_new = df_day_new[df_day_new['Full Time'] >= '2018-05-06'] # 2018-05-06 is 8 weeks prior to 2018-07-01 
df_day_new = df_day_new.drop(columns=['Entries'])
df_day_new = df_day_new.rename(index=str, columns={"Entries_diff": "Entries_per_day"})
df_day_new 

,C/A,Entries_per_day,Full Time,SCP,Station
5,A006,950.0,2018-05-06,00-00-00,5 AV/59 ST
6,A006,704.0,2018-05-07,00-00-00,5 AV/59 ST
7,A006,1602.0,2018-05-08,00-00-00,5 AV/59 ST
8,A006,1728.5,2018-05-09,00-00-00,5 AV/59 ST
9,A006,1721.0,2018-05-10,00-00-00,5 AV/59 ST
10,A006,1638.0,2018-05-11,00-00-00,5 AV/59 ST
11,A006,1706.5,2018-05-12,00-00-00,5 AV/59 ST
12,A006,913.5,2018-05-13,00-00-00,5 AV/59 ST
13,A006,701.5,2018-05-14,00-00-00,5 AV/59 ST
14,A006,1550.0,2018-05-15,00-00-00,5 AV/59 ST


In [103]:
df_day_new.to_csv('Entries_per_day.csv')